## Atomistic Molecular Dynamics: Bulk Liquids - SPC/E Water

### Build the system

In [ ]:
import mbuild as mb

class H2O(mb.Compound):
    def __init__(self):
        super(H2O, self).__init__()
        mb.load('utils/h2o.pdb', compound=self)

In [ ]:
water = H2O()
water.visualize()

In [ ]:
class WaterBox(mb.Compound):
    """An box of water."""
    def __init__(self, n_molecules):
        """Initialize an WaterBox Compound.

        Parameters
        ----------
        n_molecules : int
            Number of molecules to place in the box
        """
        super(WaterBox, self).__init__()
        
        # Create a prototype for water using the class above
        water = H2O()

        # Fill a box with water at 1000 kg/m^3
        box_of_waters = mb.fill_box(compound=water, n_compounds=n_molecules,
                                    density=1000)
        
        # Rename all molecules to `water`, this speeds up the atom-typing process
        for child in box_of_waters.children:
            child.name = 'Water'
        self.add(box_of_waters)

In [ ]:
water_box = WaterBox(500)
water_box.visualize()

In [ ]:
water_box.save('spce.gro', overwrite=True)
water_box.save('spce.top', forcefield_files='spce.xml', overwrite=True, residues='Water')

### Run the simulation

Equilibrate

In [ ]:
!gmx grompp -v -f npt-eq.mdp -c spce.gro -p spce.top -o npt-eq-spce

In [ ]:
!gmx mdrun -s npt-eq-spce.tpr -o -x -deffnm npt-eq-spce

View trajectory

Plot volume, temperature, and energy

#### NPT

In [ ]:
!gmx grompp -v -f npt-prod.mdp -c npt-eq-spce.gro -p spce.top -o npt-prod-spce

In [ ]:
!gmx mdrun -s npt-prod-spce.tpr -o -x -deffnm npt-prod-spce

View trajectory

Plot volume, temperature, and density